## DATA CLEANUP

In [1]:
####################################################################################
# Notebook to be used for cleaning up the raw data files,  special values, etc.    # 
####################################################################################

In [2]:
# Import libraries
import numpy as np
import pandas as pd
import os
import re
import datetime
from dateutil import parser

In [3]:
# Create a set of the raw data and processed files name
# CONFIG NEEDED: Uncomment only the specific files to be processed on your node

csv_files = {
 '02-14-2018.csv': '02-14-2018-bruteforce-ftp-ssh.csv',
 '02-15-2018.csv': '02-15-2018-dos-goldeneye-slowloris.csv',
 '02-16-2018.csv': '02-16-2018-dos-slowhttp-hulk.csv',
 '02-21-2018.csv': '02-21-2018-ddos-loic-udp.csv',
 '02-22-2018.csv': '02-22-2018-bruteforce-webxss.csv',
 '02-23-2018.csv': '02-23-2018-bruteforce-webxss-sql.csv',
 '02-28-2018.csv': '02-28-2018-infiltration.csv',
 '03-01-2018.csv': '03-01-2018-botnet.csv',
 '03-02-2018.csv': '03-02-2018-infiltration.csv',
 '02-20-2018.csv': '02-20-2018-ddos-loic-tcp.csv'   # WARNING: 4GB FILE.
}

In [4]:
# Set the folder name for raw data and processed files under the project directory
# CONFIG NEEDED: Change the './data' and 'processed' to what you named your directories
# Raw Data Files Location: final_project/data
# Processed Data Files Location: final_project/data/processed

rawdata_path = './data'
processed_path = os.path.join(rawdata_path, 'processed')

In [5]:
# Remove duplicate headers
def remove_headers(f):    
    return f[~f['Dst Port'].str.contains('Dst Port', na=False)]

In [6]:
# Replace 'Infinity' values with 'inf'
def replace_infinity(f):
    return f.replace('Infinity', 'inf', inplace=True)

In [7]:
# Drop rows that have 'Infinity', 'infinity', or 'inf' as value
def drop_infinity(f):
    return f[~f['flow_byts_s'].str.contains('Infinity|inf', na=False)]

In [8]:
# Drop rows that have 'NaN' as label value
def drop_nan(f):
    return f[~f['label'].str.contains('NaN', na=False)]

In [9]:
column_name_regex = re.compile(r"\W", re.IGNORECASE)

# Clean (spaces, special characters, etc.) column headers and lower case 
def remove_non_word_chars_from_column_names(f):
    return [column_name_regex.sub('_', c.lower()) for c in df.columns]

In [10]:
# Change datetime string to epochs from the date 01/01/1970
def change_datetime_epochs(f):
    # Note: The date time conversion in Pandas is extremely slow.....
    #for index, rows in f.iterrows():
    #    dt = parser.parse(rows["timestamp"])
    #    epochs = (dt - datetime.datetime(1970, 1, 1)).total_seconds()
    #    rows["timestamp"] = str(epochs)

    f['timestamp'] = pd.to_datetime(f['timestamp'])
    f['timestamp'] = f['timestamp'].astype('int64')//1e9
    return f

In [11]:
# Create folder for processed files if none exists
if not os.path.exists(processed_path):
    os.mkdir(processed_path)    

# Process the list of files specified
for f, out in csv_files.items():
    file_path = os.path.join(rawdata_path, f)
    output_path = os.path.join(processed_path, out)

    print('reading:', file_path)
    # One of the data files has four extra columns which need to be dropped. Checks each file for extra columns.
    df = pd.read_csv(file_path, dtype=str).drop(columns=['Flow ID', 'Src IP', 'Dst IP', 'Src Port'], errors='ignore')

    # Clean up the data files
    print('processing:', file_path)
    df = remove_headers(df)
    df.columns = remove_non_word_chars_from_column_names(df)
    df = drop_infinity(df)
    df = drop_nan(df)
    df = change_datetime_epochs(df)

    # Write it as a cleaned file in the processed directory
    print('writing:', output_path)
    df.to_csv(output_path, index=False)

reading: ./data/02-14-2018.csv
processing: ./data/02-14-2018.csv
writing: ./data/processed/02-14-2018-bruteforce-ftp-ssh.csv
reading: ./data/02-15-2018.csv
processing: ./data/02-15-2018.csv
writing: ./data/processed/02-15-2018-dos-goldeneye-slowloris.csv
reading: ./data/02-16-2018.csv
processing: ./data/02-16-2018.csv
writing: ./data/processed/02-16-2018-dos-slowhttp-hulk.csv
reading: ./data/02-21-2018.csv
processing: ./data/02-21-2018.csv
writing: ./data/processed/02-21-2018-ddos-loic-udp.csv
reading: ./data/02-22-2018.csv
processing: ./data/02-22-2018.csv
writing: ./data/processed/02-22-2018-bruteforce-webxss.csv
reading: ./data/02-23-2018.csv
processing: ./data/02-23-2018.csv
writing: ./data/processed/02-23-2018-bruteforce-webxss-sql.csv
reading: ./data/02-28-2018.csv
processing: ./data/02-28-2018.csv
writing: ./data/processed/02-28-2018-infiltration.csv
reading: ./data/03-01-2018.csv
processing: ./data/03-01-2018.csv
writing: ./data/processed/03-01-2018-botnet.csv
reading: ./data/0

In [12]:
############################################################################################
# Cells below this are only needed if you want to test if the files were created correctly #
# Comment/Uncomment as needed
############################################################################################

In [13]:
print(os.listdir(processed_path))  # Print list of files in the processed directory

['03-01-2018-botnet.csv', '03-02-2018-infiltration.csv', '02-28-2018-infiltration.csv', '02-22-2018-bruteforce-webxss.csv', '02-16-2018-dos-slowhttp-hulk.csv', '02-14-2018-bruteforce-ftp-ssh.csv', '02-15-2018-dos-goldeneye-slowloris.csv', 'models', '02-21-2018-ddos-loic-udp.csv', '.ipynb_checkpoints', '02-23-2018-bruteforce-webxss-sql.csv', '02-20-2018-ddos-loic-tcp.csv']


In [14]:
# Read a sample file and check label counts
# CONFIG NEEDED: Change file name to the file you want to check
df = pd.read_csv("./data/processed/03-02-2018-infiltration.csv")
df['label'].value_counts()

Benign    760892
Bot       286191
Name: label, dtype: int64

In [15]:
# Check if the data types were detected and appropriate, file size, # of rows, # of columns, etc.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1047083 entries, 0 to 1047082
Data columns (total 80 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   dst_port           1047083 non-null  int64  
 1   protocol           1047083 non-null  int64  
 2   timestamp          1047083 non-null  float64
 3   flow_duration      1047083 non-null  int64  
 4   tot_fwd_pkts       1047083 non-null  int64  
 5   tot_bwd_pkts       1047083 non-null  int64  
 6   totlen_fwd_pkts    1047083 non-null  int64  
 7   totlen_bwd_pkts    1047083 non-null  float64
 8   fwd_pkt_len_max    1047083 non-null  int64  
 9   fwd_pkt_len_min    1047083 non-null  int64  
 10  fwd_pkt_len_mean   1047083 non-null  float64
 11  fwd_pkt_len_std    1047083 non-null  float64
 12  bwd_pkt_len_max    1047083 non-null  int64  
 13  bwd_pkt_len_min    1047083 non-null  int64  
 14  bwd_pkt_len_mean   1047083 non-null  float64
 15  bwd_pkt_len_std    1047083 non-n

In [16]:
df.head()

,dst_port,protocol,timestamp,flow_duration,tot_fwd_pkts,tot_bwd_pkts,totlen_fwd_pkts,totlen_bwd_pkts,fwd_pkt_len_max,fwd_pkt_len_min,...,fwd_seg_size_min,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label
0,443,6,1.517648e+09,141385,9,7,553,3773.0,202,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,49684,6,1.517648e+09,281,2,1,38,0.0,38,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,443,6,1.517648e+09,279824,11,15,1086,10527.0,385,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,443,6,1.517648e+09,132,2,0,0,0.0,0,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,443,6,1.517648e+09,274016,9,13,1285,6141.0,517,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
